The Python script below performs the following configuration tasks:

Initializes a SparkSession with the necessary .jar files for Snowflake integration.

Loads the Snowflake JDBC and Spark-Snowflake connector using the spark.jars property.

Sets up the Spark context for PySpark operations.

Prepares the environment to read/write between Spark and Snowflake.

In [6]:
#Importing all necessary classes and modules.
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import findspark

#Initalizing the findspark module to recognize the spark installed on my system
findspark.init()

# Initialize Spark Session
spark = SparkSession.builder.appName("SpotifyDataNormalization").config("spark.jars", r"C:\spark-3.5.5-bin-hadoop3\jars\snowflake-jdbc-3.23.2.jar,C:\spark-3.5.5-bin-hadoop3\jars\spark-snowflake_2.12-3.1.1").getOrCreate()

#Setting snowflake credentials.
snowflakeConfig = {
    "sfURL": "https://cexmrap-kb36509.snowflakecomputing.com",
    "sfUser": os.getenv("SNOWFLAKE_USER1"),
    "sfPassword": os.getenv("SNOWFLAKE_PASSWORD"),
    "sfDatabase": os.getenv("SNOWFLAKE_DB1"),
    "sfSchema": os.getenv("SNOWFLAKE_SCHEMA1"),
    "sfWarehouse": os.getenv("SNOWFLAKE_WAREHOUSE1"),
    "sfRole": os.getenv("SNOWFLAKE_ROLE1")
}


The Python script below performs the following tasks:

Converts the contents of the played_at field from Unix Timestamp to a readable Date and Time format.

Writes the transformed data to a CSV file.

The CSV file will be overwritten each time the script is run.

In [13]:
# Read data into a dataframe
df = spark.read.option("multiline", "true").json("all_recent_tracks.json")

# Convert timestamp to normal date and time 
df = df.withColumn("played_at", F.from_unixtime(F.col("played_at") / 1000, "yyyy-MM-dd HH:mm:ss"))

# Creating A CSV file of the transformed data
df.coalesce(1).write.option("header", "true").mode("overwrite").csv("clean_all_tracks.csv")

# Load the transformed data into snowflake database
df.write.format("snowflake").options(**snowflakeConfig).option("dbtable", "all_tracks").option("sfTableCreation", "CREATE_IF_NEEDED").mode("append").save()

df.show(truncate=False)

Py4JJavaError: An error occurred while calling o324.save.
: net.snowflake.client.jdbc.SnowflakeReauthenticationRequest: Authentication token has expired.  The user must authenticate again.
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionSub(SnowflakeUtil.java:168)
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionIncludingReauth(SnowflakeUtil.java:99)
	at net.snowflake.client.core.SessionUtil.renewTokenRequest(SessionUtil.java:1260)
	at net.snowflake.client.core.SessionUtil.renewSession(SessionUtil.java:1162)
	at net.snowflake.client.core.SFSession.renewSession(SFSession.java:928)
	at net.snowflake.client.core.SFStatement.executeHelper(SFStatement.java:508)
	at net.snowflake.client.core.SFStatement.executeQueryInternal(SFStatement.java:211)
	at net.snowflake.client.core.SFStatement.executeQuery(SFStatement.java:145)
	at net.snowflake.client.core.SFStatement.execute(SFStatement.java:779)
	at net.snowflake.client.core.SFStatement.exec

This script, although commented out, plays a crucial role in some of our projects. Its intended purpose is to move the necessary CSV file from the folder created by PySpark after the transformation process. The goal is to take the CSV file containing the transformed data and move it to a specific location.

Why is it commented out?
Some may wonder why this script is commented. The reason is that the name of the file changes each time the script is re-run. Because PySpark generates a new file with a unique name (e.g., part-00001-...), the path of the file keeps changing. Given that the data is intended to keep growing and the file name is dynamic, it wouldn't be feasible to hard-code the path or attempt to manually move the file. This makes it challenging to automate without first addressing the changing file names.



In [ ]:
""" import shutil
# Move the part file to desired location
part_file = "clean_tracks.csv\part-00000-e767b6d4-f389-44a7-91b0-dec958759f38-c000.csv"
destination = "clean_recent_tracks.csv"
shutil.move(part_file,destination) """


<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\IFEOMA\AppData\Local\Temp\ipykernel_9832\441510881.py:1: SyntaxWarning: invalid escape sequence '\p'
  """ import shutil


' import shutil\n# Move the part file to desired location\npart_file = "clean_tracks.csv\\part-00000-e767b6d4-f389-44a7-91b0-dec958759f38-c000.csv"\ndestination = "clean_recent_tracks.csv"\nshutil.move(part_file,destination) '

The script below performs the following tasks:

Trims down the track ID: It extracts only the numbers from the full track ID, which might originally look like spotify:track:{id}. This operation helps focus on the core track ID, removing the unnecessary prefix.

Splitting and Indexing: The splitting of the track ID and indexing is used as an exercise to improve skills in working with string manipulation.

Converts song duration: The script converts song durations from milliseconds to minutes and seconds for easier readability.

Drops the link column: Since the link column was filled with null values, it was deemed unnecessary and dropped to clean up the dataset.

In [15]:
#Loading the raw file(json) into the second dataframe 
df2 = spark.read.option("multiline", "true").json("all_top_tracks.json")

# Split by ":" and get just the id which is the last part
df2 = df2.withColumn("id", F.split("id", ":").getItem(2))

#Set the duration in milliseconds to nomral minutes and seconds format
#Convert to minutes
df2 = df2.withColumn("minutes", F.floor(F.col("duration") / 60000))
#Get the seconds
df2= df2.withColumn("seconds", F.floor((F.col("duration") % 60000) / 1000))
#Join the two of them together
df2 = df2.withColumn( "duration",F.concat_ws(":", F.col("minutes"), F.lpad(F.col("seconds").cast("string"), 2, "0")))
#Removing the link column
df2 = df2.drop("link")

#Converting the dataframe into a csv file 
df2.coalesce(1).write.option("header", "true").mode("overwrite").csv("clean_all_top_tracks.csv")

# Load the transformed data into snowflake database
df2.write.format("snowflake").options(**snowflakeConfig).option("dbtable", "top_tracks").option("sfTableCreation", "CREATE_IF_NEEDED").mode("append").save()

df2.show(truncate=False)

Py4JJavaError: An error occurred while calling o395.save.
: net.snowflake.client.jdbc.SnowflakeReauthenticationRequest: Authentication token has expired.  The user must authenticate again.
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionSub(SnowflakeUtil.java:168)
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionIncludingReauth(SnowflakeUtil.java:99)
	at net.snowflake.client.core.SessionUtil.renewTokenRequest(SessionUtil.java:1260)
	at net.snowflake.client.core.SessionUtil.renewSession(SessionUtil.java:1162)
	at net.snowflake.client.core.SFSession.renewSession(SFSession.java:928)
	at net.snowflake.client.core.SFStatement.executeHelper(SFStatement.java:508)
	at net.snowflake.client.core.SFStatement.executeQueryInternal(SFStatement.java:211)
	at net.snowflake.client.core.SFStatement.executeQuery(SFStatement.java:145)
	at net.snowflake.client.core.SFStatement.execute(SFStatement.java:779)
	at net.snowflake.client.core.SFStatement.exec

The script below performs the following tasks:

Retrieves all contents of the genre column: The genre column in the raw JSON file is an array containing multiple genre values for each track.

Extracts all genre values: The script extracts and processes each genre from the array, ensuring all the different genres associated with a track are retrieved.

In [14]:
#Loading the raw file(json) into the third dataframe 
df3 = spark.read.option("multiline", "true").json("all_top_artists.json")

#Getting all the contents of the genre array
df3 = df3.withColumn("genre", F.explode("Genre"))

#Converting the dataframe into a csv file 
df3.coalesce(1).write.option("header", "true").mode("overwrite").csv("clean_all_top_artists.csv")

# Load the transformed data into snowflake database
df3.write.format("snowflake").options(**snowflakeConfig).option("dbtable", "top_artist").option("sfTableCreation", "CREATE_IF_NEEDED").mode("append").save()

df3.show(truncate=False)

Py4JJavaError: An error occurred while calling o349.save.
: net.snowflake.client.jdbc.SnowflakeReauthenticationRequest: Authentication token has expired.  The user must authenticate again.
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionSub(SnowflakeUtil.java:168)
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionIncludingReauth(SnowflakeUtil.java:99)
	at net.snowflake.client.core.SessionUtil.renewTokenRequest(SessionUtil.java:1260)
	at net.snowflake.client.core.SessionUtil.renewSession(SessionUtil.java:1162)
	at net.snowflake.client.core.SFSession.renewSession(SFSession.java:928)
	at net.snowflake.client.core.SFStatement.executeHelper(SFStatement.java:508)
	at net.snowflake.client.core.SFStatement.executeQueryInternal(SFStatement.java:211)
	at net.snowflake.client.core.SFStatement.executeQuery(SFStatement.java:145)
	at net.snowflake.client.core.SFStatement.execute(SFStatement.java:779)
	at net.snowflake.client.core.SFStatement.exec